# Steps for all of us

Choose a dataset that you want to use. 

You may do whatever steps you think necessary for building the best classifier.

Take the data you chose and do whatever massaging you think is necessary: standardizing, scaling, feature engineering/ transforming, feature selection, etc.  

Build a classifier however you see fit. You may want build one and tweak the paramters manually or use some sort of grid search to look through all possible parameters. 

Remember: The same model built on the massaged data may perform better than if the data was untouched. It may be more conveniant to chose a standared massaging pipeline and tweak a model to that data.

After you massage your data, follow these steps:

if you want to balance your target (which you should) follow along these lines:

### build your features data and target data

- X = df.drop(columns = "whatever your target name is")
- y = df"whatever your target name is"

### split the data into training and testing

- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2019)

### create the oversampled data to train on 

- oversampler = SMOTE(random_state = 2019)
- X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

### Put the oversampled data back into a dataframe

- X_train_oversampled = pd.DataFrame(X_train_oversampled, columns = X_train.columns)
- y_train_oversampled = pd.Series(y_train_oversampled)

### Build your classifier here. As an example:

- xgb_clf = xgb.XGBClassifier(max_depth=5, n_estimators=100, colsample_bytree=0.3, learning_rate=0.1, n_jobs=-1)

 
### Fit to the oversampled data; this will train the classifier on the oversampled data

- xgb_clf.fit(X_train_oversampled, y_train_oversampled)

### Use 5-fold cross validation to see how well the classfier you built is doing on test data. 
Some points: you have to substitute your classifer name in the cross_val_score function 

- kfold = KFold(n_splits=5, random_state=2019)
- results = cross_val_score(xgb_clf, X_test, y_test, cv=kfold, scoring = 'f1')


## It may be best to keep all of your models you built; have a log of them to see their scores and keep a record of your process of building your data. 



In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from Modules import *
sns.set()
%matplotlib inline
import imblearn


### read in the full sequential data

In [3]:
df = pd.read_csv('Sequential_data_trimmed.csv')
y = df['Y']

df.head().T

,0,1,2,3,4
AGE,24.000000,26.000000,34.000000,37.000000,57.000000
PAY_1,2.000000,-1.000000,0.000000,0.000000,-1.000000
PAY_2,2.000000,2.000000,0.000000,0.000000,0.000000
PAY_3,-1.000000,0.000000,0.000000,0.000000,-1.000000
PAY_4,-1.000000,0.000000,0.000000,0.000000,0.000000
PAY_5,-2.000000,0.000000,0.000000,0.000000,0.000000
PAY_6,-2.000000,2.000000,0.000000,0.000000,0.000000
Y,1.000000,1.000000,0.000000,0.000000,0.000000
SEX_Female,1.000000,1.000000,1.000000,1.000000,0.000000
SEX_Male,0.000000,0.000000,0.000000,0.000000,1.000000


In [4]:
#split my data
X = df.drop(columns = 'Y')
y = df['Y']

In [6]:
#split into training and testing

from sklearn.model_selection import train_test_split, KFold, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 2019, test_size = .2)

In [7]:
from imblearn.over_sampling import SMOTE

#create my oversampled data to train on
oversampler = SMOTE(random_state = 2019)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

In [8]:
#Put the oversampled data back into a dataframe
X_train_oversampled = pd.DataFrame(X_train_oversampled, columns = X_train.columns)
y_train_oversampled = pd.Series(y_train_oversampled)

In [15]:
# build my classifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
clf = LogisticRegression()

In [11]:
#Fit to the oversampled data; this will train the classifier on the oversampled data
clf.fit(X_train_oversampled, y_train_oversampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [12]:
#Use 5-fold cross validation to see how well the classfier you built is doing on test data.

kfold = KFold(n_splits=5, random_state=2019)
results = cross_val_score(clf, X_test, y_test, cv=kfold, scoring = 'f1')

print(f"5-fold cross-validation results: {np.mean(results)}")

5-fold cross-validation results: 0.370941751872358


In [13]:
#tweak my classifer

clf2 = LogisticRegression(C = .5)

clf2.fit(X_train_oversampled, y_train_oversampled)

results = cross_val_score(clf2, X_test, y_test, cv=kfold, scoring = 'f1')

print(f"5-fold cross-validation results: {np.mean(results)}")

5-fold cross-validation results: 0.3607249006015033


In [ ]:
clf_cv = LogisticRegressionCV(n_jobs = -1)

clf_cv.fit(X_train_oversampled, y_train_oversampled)

results = cross_val_score(clf, X_test, y_test, cv=kfold, scoring = 'f1')

print(f"5-fold cross-validation results: {np.mean(results)}")